Consider the boston house-prices dataset

In [2]:
from sklearn.datasets import load_boston
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt

In [4]:
boston = load_boston()
# training inputs, 506 houses with 13-dim information for each house
X_training = boston.data
print(X_training)
# training outputs, 506 house-prices
Y_training = boston.target

[[6.3200e-03 2.7310e-02 2.7290e-02 ... 6.0760e-02 1.0959e-01 4.7410e-02]
 [1.8000e+01 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.3100e+00 7.0700e+00 7.0700e+00 ... 1.1930e+01 1.1930e+01 1.1930e+01]
 ...
 [1.5300e+01 1.7800e+01 1.7800e+01 ... 2.1000e+01 2.1000e+01 2.1000e+01]
 [3.9690e+02 3.9690e+02 3.9283e+02 ... 3.9690e+02 3.9345e+02 3.9690e+02]
 [4.9800e+00 9.1400e+00 4.0300e+00 ... 5.6400e+00 6.4800e+00 7.8800e+00]]


We would like to use the regularized least-squares regularization method to find a function $f(x) = w_0 + \sum_{i=1}^{13}w_ix_i$ to model the house prices:
$${\rm min} \sum_{j}(f(x_j)-y_j)^{2} + \mu \|w\|_{l^2}^{2}$$
where $w = (w_0, w_1, \cdots, w_{13})$. For a given regularization parameter $\mu$, we already know how to solve the above problem.

Use 5-fold cross validation method to find the optimal parameter $\mu$ in $\{10, 1, 0.1, 0.01, 0.001\}$ with respect to the mean square error
$$MSE = \sum_j (f(x_j)-y_j)^{2}$$
Note that when randomly dividing the data into 5 groups, they do not need to have exactly the same size. For example, you could have 5 groups with sizes 101, 101, 101, 101, and 102

First, we use general way to solve this optimization problem

Let $\mathbf{w} := (w_0, w_1, \cdots, w_{13})^{T}$, $\mathbf{y} = (y_1, y_2, \cdots, y_N)$
\begin{equation}
\mathbf{A}: = 
\begin{bmatrix}
1 & \mathbf{x}_1\\
1 & \mathbf{x}_2\\
\vdots & \vdots \\
1 & \mathbf{x}_N
\end{bmatrix}
\end{equation}

Then $F(\mathbf{w}): =  \sum_{j}(f(x_j)-y_j)^{2} + \mu \|w\|_{l^2}^{2}$ can be write as the matrix-vector form
$$F(\mathbf{w}) = (\mathbf{Aw-y})^{T}(\mathbf{Aw-y})+ \mu \mathbf{w}^{T}\mathbf{w}$$

The gradient of F is
$$\nabla F (\mathbf{w}) = 2 \mathbf{A}^{T}(\mathbf{Aw-y})+ 2\mu \mathbf{w}$$

Thus 
$$\mathbf{w}  = (\mathbf{A^TA+\mu I})^{-1}\mathbf{A^Ty}$$

In [5]:
# consider 5-fold cross validation
index_1=np.random.choice(506, 101, replace=False)
left_index1 = [x for x in range(506) if x not in index_1]
index_2 = np.random.choice(left_index1, 101, replace=False)
left_index2 = [x for x in left_index1 if x not in index_2]
index_3 = np.random.choice(left_index2, 101, replace=False)
left_index3 = [x for x in left_index2 if x not in index_3]
index_4 = np.random.choice(left_index3, 101, replace=False)
index_5 = [x for x in left_index3 if x not in index_4]
index = [index_1, index_2, index_3, index_4, index_5]

Mu = np.array([10, 1, 0.1, 1e-2, 1e-3])

def crossed(X_training,Y_training, ind, mu):
    X_test = X_training[ind,:]
    Y_test = Y_training[ind]
    left_index = [x for x in range(506) if x not in ind]
    X_train = X_training[left_index,:]
    Y_train = Y_training[left_index]
    
    n = np.shape(X_train)[0]
    A = np.block([np.ones([n,1]), X_train])
    y = np.reshape(Y_train,[np.size(Y_train),1])
    
    dim = np.shape(A)[1]
    w = LA.solve(np.dot(A.T, A) + mu * np.eye(dim), np.dot(A.T, y))
    #print(w,'\n','***********************************')
    
    return w

def MSErr(X_training, Y_training, ind, w):
    X_test = X_training[ind,:]
    Y_test = Y_training[ind]    
    n = np.shape(X_test)[0]
    A = np.block([np.ones([n,1]), X_test])
    y = np.reshape(Y_test,[np.size(Y_test),1])
    
    #error = LA.norm(np.dot(A,w)-y)
    error = np.dot((np.dot(A,w)-y).T, np.dot(A,w)-y)
    return error

# 5-fold cross validation     
def cross_validation(X_training, Y_training, index , mu):
    Err = 0
    for i in range(5):
        ind = index[i]
        w = crossed(X_training,Y_training, ind, mu)
        mserr = MSErr(X_training, Y_training, ind, w)
        Err = Err + mserr
    return Err
    
# Find proper mu
def Proper_mu(X_training, Y_training, index, Mu):
    n = np.size(Mu)
    Error = np.zeros(n)
    for i in range(n):
        mu = Mu[i]
        Err = cross_validation(X_training, Y_training, index , mu)
        Error[i] = Err
    k = np.argmin(Error)
    pro_mu = Mu[k]
    return pro_mu, Error[k]

In [101]:
pro_mu, Error = Proper_mu(X_training, Y_training, index, Mu)

In [102]:
print("the most proper mu is:",pro_mu,"\n", "the MSE error: ", Error)

the most proper mu is: 0.01 
 the MSE error:  11865.214444764742


I have run the code many times, sometimes the most suitable is 0.1, sometimes is 0.01, sometimes is 0.001. So I think [0.1,0.01,0.001] can be used in this project.

### The MSE error is so large, one possible reason is the linear model is not suitable for the data.

In [7]:
print(X_training)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]


In [8]:
import pandas as pd

df = pd.DataFrame(data=X_training)
df.to_csv('X_trainging.csv',index=False)
df1 = pd.DataFrame(data=Y_training)
df1.to_csv('Y_trainging.csv',index=False)